In [11]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [12]:
import sys
import glob
import os
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from sklearn.model_selection import ParameterGrid
from pathlib import Path
import pickle


In [13]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]

In [15]:
sats_games = []
PREFIXES = [
    # 'oct16_encumbereddeviations',
    # 'sep20',
    # 'dec11_bc',
    # 'dec13b_bc',
    # 'jan19'
    # 'jan19boring'
    'feb4_1t', 'feb4_2t', 'feb4_3t', 'feb4_4t', # feb4 games: 1-4 types
]

for prefix in PREFIXES:
    sats_games += [f'{prefix}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{prefix}/*')]
sats_games = list(set(sats_games))
# can apply other filters like:
# sats_games = [s for s in sats_games if ('hide' not in s) and '1000' in s and 'risk_averse' not in s]
# sats_games = sats_games[:1] # debug
print(len(sats_games))
print(sats_games)

80
['feb4_4t/feb4_4t_4_base_dev1000_rho1_t4', 'feb4_3t/feb4_3t_2_base_dev1000_rho0_t3_tie_break', 'feb4_1t/feb4_1t_1_base_dev1000_rho1_t1_tie_break', 'feb4_1t/feb4_1t_0_base_dev1000_rho0_t1', 'feb4_2t/feb4_2t_2_base_dev1000_rho1_t2', 'feb4_3t/feb4_3t_0_base_dev1000_rho1_t3', 'feb4_1t/feb4_1t_4_base_dev1000_rho0_t1_tie_break', 'feb4_3t/feb4_3t_4_base_dev1000_rho0_t3_tie_break', 'feb4_4t/feb4_4t_2_base_dev1000_rho0_t4', 'feb4_2t/feb4_2t_1_base_dev1000_rho1_t2', 'feb4_1t/feb4_1t_4_base_dev1000_rho0_t1', 'feb4_4t/feb4_4t_1_base_dev1000_rho1_t4', 'feb4_4t/feb4_4t_1_base_dev1000_rho1_t4_tie_break', 'feb4_2t/feb4_2t_1_base_dev1000_rho0_t2', 'feb4_1t/feb4_1t_1_base_dev1000_rho0_t1_tie_break', 'feb4_2t/feb4_2t_4_base_dev1000_rho1_t2_tie_break', 'feb4_3t/feb4_3t_4_base_dev1000_rho0_t3', 'feb4_3t/feb4_3t_3_base_dev1000_rho0_t3_tie_break', 'feb4_2t/feb4_2t_0_base_dev1000_rho1_t2_tie_break', 'feb4_3t/feb4_3t_1_base_dev1000_rho1_t3_tie_break', 'feb4_4t/feb4_4t_1_base_dev1000_rho0_t4', 'feb4_4t/feb4_

In [16]:
pgrid = ParameterGrid([{
    'game': [g for g in sats_games]# if 'hide' not in g and 'imperfect' not in g]
}])
print(len(pgrid))

80


In [18]:
num_iters = 999_000_000
time_limit_seconds = int(3600 * 1)
N_SEEDS = 5 # Number of seeds of each experiment to launch
extra_name = ''
overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 1000 --time_limit_seconds {time_limit_seconds} --use_wandb true'      

for config in pgrid:
    for alg_config, alg_name in [('cfr_10', 'cfr'), ('ppo_jun8_23', 'ppo')]:
        dispatch.dispatch_experiments(alg_config, base_job_name='feb4_v1', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg=alg_name)

Submitted batch job 1080348
Submitted batch job 1080349
Submitted batch job 1080350
Submitted batch job 1080351
Submitted batch job 1080352
Submitted batch job 1080353
Submitted batch job 1080354
Submitted batch job 1080355
Submitted batch job 1080356
Submitted batch job 1080357
Submitted batch job 1080358
Submitted batch job 1080359
Submitted batch job 1080360
Submitted batch job 1080361
Submitted batch job 1080362
Submitted batch job 1080363
Submitted batch job 1080364
Submitted batch job 1080365
Submitted batch job 1080366
Submitted batch job 1080367
Submitted batch job 1080368
Submitted batch job 1080369
Submitted batch job 1080370
Submitted batch job 1080371
Submitted batch job 1080372
Submitted batch job 1080373
Submitted batch job 1080374
Submitted batch job 1080375
Submitted batch job 1080376
Submitted batch job 1080377
Submitted batch job 1080378
Submitted batch job 1080379
Submitted batch job 1080380
Submitted batch job 1080381
Submitted batch job 1080382
Submitted batch job 

In [19]:
# other example jobs below

In [6]:
# ### Just GOF CFR
# num_iters = 999_000_000
# time_limit_seconds = int(3600 * 1)
# for config in pgrid:
#     CONFIG = 'cfr' # The YML network config folder for the main jobs
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 1800 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 50 --time_limit_seconds {time_limit_seconds} --use_wandb true'      
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jan18_cfr_v6_seeds_v3', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr', start_seed=200)

In [10]:
num_iters = 999_000_000
time_limit_seconds = int(3600 * 1)
for config in pgrid:
    CONFIG = 'cfr_10' # The YML network config folder for the main jobs
    N_SEEDS = 5 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 250 --time_limit_seconds {time_limit_seconds} --use_wandb true'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='feb4_v1', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 1079913


In [10]:
num_iters = 999_000_000
time_limit_seconds = int(3600 * 1 / 60)
for config in pgrid:
    CONFIG = 'ppo_jun8_23' # The YML network config folder for the main jobs
    N_SEEDS = 5 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 250 --time_limit_seconds {time_limit_seconds} --use_wandb true'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='feb3_test_short_v4', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='ppo')

Submitted batch job 1080222
Submitted batch job 1080223
Submitted batch job 1080224
Submitted batch job 1080225
Submitted batch job 1080226
Submitted batch job 1080227
Submitted batch job 1080228
Submitted batch job 1080229
Submitted batch job 1080230
Submitted batch job 1080231
Submitted batch job 1080232
Submitted batch job 1080233
Submitted batch job 1080234
Submitted batch job 1080235
Submitted batch job 1080236
Submitted batch job 1080237
Submitted batch job 1080238
Submitted batch job 1080239
Submitted batch job 1080240
Submitted batch job 1080241
Submitted batch job 1080242
Submitted batch job 1080243
Submitted batch job 1080244
Submitted batch job 1080245
Submitted batch job 1080246
Submitted batch job 1080247
Submitted batch job 1080248
Submitted batch job 1080249
Submitted batch job 1080250
Submitted batch job 1080251
Submitted batch job 1080252
Submitted batch job 1080253
Submitted batch job 1080254
Submitted batch job 1080255
Submitted batch job 1080256
Submitted batch job 

In [25]:
!squeue -u $USER --format="%.18i %.90j %.2t .%.10M%.6D %R"

             JOBID                                                                                       NAME ST .      TIME NODES NODELIST(REASON)
           1079910                                                                                    jupyter  R .   1:45:52     1 ada21
           1080011 jan12_repro_4t_jan12_repro_4t_1_base_dev1000_rho0_t4-ppo_jun8_23ppo_76-100_feb3_test_short  R .      1:08     1 vickrey07
           1080012 jan12_repro_4t_jan12_repro_4t_1_base_dev1000_rho0_t4-ppo_jun8_23ppo_76-101_feb3_test_short  R .      1:08     1 vickrey07
           1080013 jan12_repro_4t_jan12_repro_4t_1_base_dev1000_rho0_t4-ppo_jun8_23ppo_76-102_feb3_test_short  R .      1:08     1 vickrey08
           1080014 jan12_repro_4t_jan12_repro_4t_1_base_dev1000_rho0_t4-ppo_jun8_23ppo_76-103_feb3_test_short  R .      1:08     1 vickrey08
           1080015 jan12_repro_4t_jan12_repro_4t_1_base_dev1000_rho0_t4-ppo_jun8_23ppo_76-104_feb3_test_short  R .      1:08     1 vickrey08
          

In [7]:
!squeue -u $USER --format="%.18i %.65j %.2t %.10M %.6D %R" | grep -v "jupyter" | awk '{print $1}'

JOBID
1080186
1080187
1080188
1080189
1080190
1080191
1080192
1080193
1080194
1080195
1080196
1080197
1080198
1080199
1080200
1080201
1080202
1080203
1080204
1080205
1080206
1080207
1080208
1080209
1080210
1080211
1080212
1080213
1080214
1080215
1080216
1080217
1080218
1080219
1080220
1080221
1080180
1080181
1080182
1080183
1080184
1080185
1080171
1080172
1080173
1080174
1080175
1080176
1080177
1080178
1080179
1080163
1080164
1080165
1080166
1080167
1080168
1080169
1080170
1080154
1080155
1080156
1080157
1080158
1080159
1080160
1080161
1080162
1080146
1080147
1080148
1080149
1080150
1080151
1080152
1080153
1080140
1080141
1080142
1080143
1080144
1080145
1080131
1080132
1080133
1080134
1080135
1080136
1080137
1080138
1080139
1080125
1080126
1080127
1080128
1080129
1080130
1080123
1080124
1080122


In [9]:
!squeue -u $USER | wc -l

102


In [17]:
def check_queue():
    !ssh borg.cs.ubc.ca '/opt/slurm/bin/squeue -u $USER --format="%.18i %.90j %.2t .%.10M %.6D %R"'


In [18]:
check_queue()

             JOBID                                                                                       NAME ST .      TIME  NODES NODELIST(REASON)
           1079910                                                                                    jupyter  R .     49:46      1 ada21
           1079915 jan12_repro_4t_jan12_repro_4t_1_base_dev1000_rho1_t4-ppo_jun8_23ppo_76-1350985_who_cares_p  R .      1:27      1 ada01
           1079913 jan12_repro_4t_jan12_repro_4t_1_base_dev1000_rho1_t4-cfr_10external_plus_linear-100_who_ca  R .     36:32      1 ada01


In [ ]:
!squeue | grep "may30" | wc -l

In [ ]:
!scancel -u ubuntu

In [20]:
!scancel  1079920 

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
Evaluation.objects.last()

In [ ]:
os.system("ssh borg.cs.ubc.ca '/opt/slurm/bin/sbatch'")

In [ ]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "eval" | awk '{print $1}' | xargs scancel

In [ ]:
from compress_pickle import dumps, loads


In [ ]:
loads?

In [ ]:
from compress_pickle.compressers.registry import get_known_compressions

In [ ]:
get_known_compressions()

In [ ]:
x = EquilibriumSolverRunCheckpoint.objects.last().policy

In [ ]:
len(x)

In [ ]:
len(dumps(pickle.loads(x), compression='bz2'))

In [ ]:
len(pickle.dumps(pickle.loads(x)))

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().walltime

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().t

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().bestresponse_set.all()

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().policy)

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().bestresponse_set.all().first().model)

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().bestresponse_set.all().last().model)